In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from keras.utils import np_utils
import numpy as np
np.random.seed(10)

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
#load datas and process

In [3]:
from keras.datasets import mnist
((x_train_image, y_train_label), (x_test_image, y_test_label)) = mnist.load_data()

In [4]:
x_train = x_train_image.reshape( len(x_train_image), x_train_image.shape[1]*x_train_image.shape[2] ).astype('float32')
x_test = x_test_image.reshape( len(x_test_image), x_test_image.shape[1]*x_test_image.shape[2] ).astype('float32')
print(x_train.shape)
print(x_test.shape)

(60000, 784)
(10000, 784)


In [5]:
x_train_normalize = x_train/255
test_images = x_test/255

In [6]:
y_train_onehot = np_utils.to_categorical(y_train_label)
test_labels = np_utils.to_categorical(y_test_label)
print(y_train_onehot.shape)
print(test_labels.shape)

(60000, 10)
(10000, 10)


In [7]:
from sklearn.model_selection import train_test_split
train_images, validation_images, train_labels, validation_labels = train_test_split(x_train_normalize, y_train_onehot, test_size=0.1, random_state=42)

In [8]:
print('train imgs:', train_images.shape, 'labels:', train_labels.shape)
print('validation imgs:', validation_images.shape, 'labels:', validation_labels.shape)
print('test imgs:', test_images.shape, 'labels:', test_labels.shape)

train imgs: (54000, 784) labels: (54000, 10)
validation imgs: (6000, 784) labels: (6000, 10)
test imgs: (10000, 784) labels: (10000, 10)


In [9]:
#define model funtions, dense, conv2d and mas_pooling

In [10]:
def layer(output_dim, input_dim, inputs, activation=None):  #dense
  W = tf.Variable(tf.random_normal([input_dim, output_dim]))
  b = tf.Variable(tf.random_normal([1, output_dim]))
  XWb = tf.matmul(inputs, W) + b
  if activation is None:
    outputs = XWb
  else:
    outputs = activation(XWb)
  return outputs

In [11]:
def dropout(h):  #dropout
  return tf.nn.dropout(h, rate=0.2)

In [12]:
def weight(shape):  #conv2d
  return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

def bias(shape):
  return tf.Variable(tf.constant(0.1, shape=shape))

def conv2d(x, W, b):
  return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')+b 

In [13]:
def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [14]:
#build MLP model
x = tf.placeholder("float", [None, 784])
h1 = layer(output_dim=1000, input_dim=784, inputs=x, activation=tf.nn.relu)
drop_h1 = dropout(h1)
h2 = layer(output_dim=1000, input_dim=1000, inputs=drop_h1, activation=tf.nn.relu)
drop_h2 = dropout(h2)
#y_predict = layer(output_dim=10, input_dim=1000, inputs=h2, activation=tf.nn.softmax)
y_predict = layer(output_dim=10, input_dim=1000, inputs=drop_h2, activation=None)

In [15]:
#training method
y_label = tf.placeholder('float', [None, 10])
loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_predict, labels=y_label))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss_function)

In [16]:
#accuracy Evaluation method
correct_prediction = tf.equal(tf.argmax(y_label, 1), tf.argmax(y_predict, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [17]:
#start training
trainEpochs = 15
batchSize = 100
totalbatch = int(len(train_images)/batchSize)
loss_list = []; epoch_list = []; accuracy_list = []
from time import time
startTime=time()

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [18]:
for epoch in range(trainEpochs):
  for i in range(totalbatch):
    batch_x = train_images[i*batchSize:(i+1)*batchSize]
    batch_y = train_labels[i*batchSize:(i+1)*batchSize]
    sess.run(optimizer, feed_dict={x: batch_x, y_label: batch_y})
  
  loss, acc = sess.run([loss_function, accuracy], feed_dict={x: validation_images, y_label: validation_labels})

  epoch_list.append(epoch)
  loss_list.append(loss)
  accuracy_list.append(acc)
  print("Train Epoch:", '%02d' %(epoch+1), "Loss=", "{:.9f}".format(loss), "Accuracy=", acc)

duration = time() - startTime
print("Training Time", duration)

Train Epoch: 01 Loss= 375.943634033 Accuracy= 0.86
Train Epoch: 02 Loss= 219.245056152 Accuracy= 0.8955
Train Epoch: 03 Loss= 158.017227173 Accuracy= 0.90566665
Train Epoch: 04 Loss= 128.949249268 Accuracy= 0.9203333
Train Epoch: 05 Loss= 107.609710693 Accuracy= 0.9238333
Train Epoch: 06 Loss= 82.915512085 Accuracy= 0.9315
Train Epoch: 07 Loss= 81.893859863 Accuracy= 0.93633336
Train Epoch: 08 Loss= 67.654747009 Accuracy= 0.937
Train Epoch: 09 Loss= 60.669635773 Accuracy= 0.9445
Train Epoch: 10 Loss= 56.973415375 Accuracy= 0.94533336
Train Epoch: 11 Loss= 58.518806458 Accuracy= 0.941
Train Epoch: 12 Loss= 50.698177338 Accuracy= 0.94816667
Train Epoch: 13 Loss= 45.198814392 Accuracy= 0.94916666
Train Epoch: 14 Loss= 41.750308990 Accuracy= 0.9561667
Train Epoch: 15 Loss= 40.184123993 Accuracy= 0.955
Training Time 16.647433519363403


In [19]:
#testing accuracy and predict
print("Accuracy:", sess.run(accuracy, feed_dict={x: test_images, y_label: test_labels}))

Accuracy: 0.9562


In [20]:
prediction_result = sess.run(tf.argmax(y_predict,1), feed_dict={x: test_images})
print(prediction_result[:10])
real_num = np.argmax(test_labels, axis=1)
print(real_num[:10])

[7 2 1 0 4 1 4 9 5 9]
[7 2 1 0 4 1 4 9 5 9]
